In [2]:
import numpy as np
import rir_generator as rir
import pandas as pd
import torch
import torch.nn as nn
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from utils.model import Geometry_estimator, Link_model
from utils.dataLoader import ISMDataset

In [48]:
class fcModel(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.fc1 = nn.Linear(4096, 2048)
        self.fc2 = nn.Linear(2048, 1024)   #1 
        self.fc3 = nn.Linear(1024, 512)   #2
        self.fc4 = nn.Linear(512, 256)     #3
        self.fc5 = nn.Linear(256, 128)    #4
        self.fc6 = nn.Linear(128, 64)      #5
        self.fc7 = nn.Linear(64, 32)       #6
        self.fc7 = nn.Linear(32, 16)       #7
        self.fc8 = nn.Linear(16, 8)      #8
        self.fc9 = nn.Linear(8, 6)  
        self.rel = nn.ReLU() 
    def forward(self, inp):
        # inp is (b,1,4096)
        # out is (b,1,6)
        inp = self.rel(self.fc1(inp))
        inp = self.rel(self.fc2(inp))
        inp = self.rel(self.fc3(inp))
        inp = self.rel(self.fc4(inp))
        inp = self.rel(self.fc5(inp))
        inp = self.rel(self.fc6(inp))
        inp = self.rel(self.fc7(inp))
        inp = self.rel(self.fc8(inp))
        inp = self.fc9(inp)
        return inp

In [10]:
mlp = nn.Sequential(
                        nn.Linear(4096, 2048), nn.ReLU(),
                        nn.Linear(2048, 1024), nn.ReLU(),   #1 
                        nn.Linear(1024, 512), nn.ReLU(),    #2
                        nn.Linear(512, 256), nn.ReLU(),     #3
                        nn.Linear(256, 128), nn.ReLU(),     #4
                        nn.Linear(128, 64), nn.ReLU(),      #5
                        nn.Linear(64, 32), nn.ReLU(),       #6
                        nn.Linear(32, 16), nn.ReLU(),       #7
                        nn.Linear(16, 8), nn.ReLU(),        #8
                        nn.Linear(8, 6)                     #9
                    ).cuda()

In [10]:
torch.randn((2,1,2)).size()

torch.Size([2, 1, 2])

In [12]:
mlp(torch.randn(5,4096).cuda())

tensor([[ 0.2924,  0.0398, -0.3574, -0.2492,  0.0020, -0.1014],
        [ 0.2924,  0.0398, -0.3573, -0.2492,  0.0020, -0.1013],
        [ 0.2926,  0.0399, -0.3572, -0.2492,  0.0021, -0.1012],
        [ 0.2925,  0.0399, -0.3572, -0.2492,  0.0021, -0.1012],
        [ 0.2926,  0.0400, -0.3572, -0.2492,  0.0022, -0.1013]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [2]:
import numpy as np

name_variant = 'fake'
epochs_st = 20
epochs_end = 45
loss_data = np.linspace(0,1,10)
np.save(f'{name_variant}_{epochs_st}_{epochs_end}_data.npy', loss_data)

In [14]:
xs = mlp 
xs(torch.randn(5,4096).cuda())

tensor([[ 0.2927,  0.0400, -0.3572, -0.2493,  0.0023, -0.1012],
        [ 0.2925,  0.0398, -0.3575, -0.2492,  0.0020, -0.1014],
        [ 0.2926,  0.0400, -0.3571, -0.2492,  0.0022, -0.1012],
        [ 0.2926,  0.0399, -0.3573, -0.2492,  0.0021, -0.1013],
        [ 0.2926,  0.0400, -0.3570, -0.2492,  0.0022, -0.1012]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [7]:
mm = torch.jit.script(mlp)
#mo = mm.type(torch.cuda.FloatTensor)
mm( torch.randn(5, 6))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/jeet/anaconda3/envs/mpa/lib/python3.10/site-packages/torch/nn/modules/container.py", line 217, in forward
    def forward(self, input):
        for module in self:
            input = module(input)
                    ~~~~~~ <--- HERE
        return input
  File "/home/jeet/anaconda3/envs/mpa/lib/python3.10/site-packages/torch/nn/modules/linear.py", line 114, in forward
    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)
               ~~~~~~~~ <--- HERE
RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x6 and 4096x2048)


In [24]:
x = np.load('./rirData/ism_400k.npy')


In [25]:
x.shape

(400000, 4105)

In [26]:
data = pd.DataFrame(x)

In [27]:
train_head = data.index[int(0.7*len(data))]
val_head = data.index[int(0.9*len(data))]
train_head, val_head

(280000, 360000)

In [28]:
train_d = data.loc[:train_head-1, :]
val_d = data.loc[train_head:val_head-1, :]
test_d = data.loc[val_head:, :]
print(f'trainData:{train_d.shape}, valData:{val_d.shape}, testData-{test_d.shape}')

trainData:(280000, 4105), valData:(80000, 4105), testData-(40000, 4105)


In [29]:
batch_size = 50
train_batches = torch.utils.data.DataLoader(ISMDataset(data=train_d), batch_size=batch_size,  num_workers=4)
val_batches = torch.utils.data.DataLoader(ISMDataset(data=val_d), batch_size=batch_size,  num_workers=4)
test_batches = torch.utils.data.DataLoader(ISMDataset(data=test_d), batch_size=batch_size,  num_workers=4)
print(f"batches: train-{len(train_batches)}, val-{len(val_batches)}, test-{len(test_batches)}")

batches: train-5600, val-1600, test-800


In [ ]:
de

In [84]:
epoch = 22
name = 'GGeo'
print( '{}_checkpoints/checkpoint_{:03d}.ckp'.format(name,epoch))

GGeo_checkpoints/checkpoint_022.ckp


In [29]:
np.random.seed(1)

In [6]:
torch.var(torch.randn((10,3,3)), dim=0)

tensor([[0.9951, 0.3938, 0.7739],
        [0.8201, 2.1408, 1.3790],
        [1.5899, 0.5538, 0.9235]])

In [5]:
torch.manual_seed(1)

s,d=torch.randn((10,6)), torch.randn((10,6)), 
mean_absolute_error(s,d, multioutput='raw_values')

array([0.89574414, 1.2751881 , 0.8364169 , 1.3237574 , 1.215802  ,
       1.1126249 ], dtype=float32)

In [7]:
torch.mean(torch.abs(s-d), axis=0)

tensor([0.8957, 1.2752, 0.8364, 1.3238, 1.2158, 1.1126])

In [8]:
mean_squared_error(s, d, multioutput='raw_values')

array([1.0732129, 2.1878397, 0.9767858, 2.4214172, 1.7687699, 1.5943434],
      dtype=float32)

In [10]:
torch.sqrt(torch.mean(torch.square(s-d), axis=0))

tensor([1.0360, 1.4791, 0.9883, 1.5561, 1.3300, 1.2627])

In [167]:
np.abs(s-d)

array([[2, 8, 1, 7, 6, 0],
       [7, 5, 6, 4, 2, 6],
       [3, 1, 2, 2, 6, 2]])

In [176]:
np.mean(np.abs(s-d), axis=0)

array([[3.9, 2.1, 3.8],
       [4.5, 1.7, 2.8],
       [1.5, 3.6, 2.3]])

In [149]:
s

array([[6, 9, 6, 1, 1, 2],
       [8, 7, 3, 5, 6, 3],
       [5, 3, 5, 8, 8, 2],
       [8, 1, 7, 8, 7, 2],
       [1, 2, 9, 9, 4, 9],
       [8, 4, 7, 6, 2, 4],
       [5, 9, 2, 5, 1, 4],
       [3, 1, 5, 3, 8, 8],
       [9, 7, 4, 8, 8, 5],
       [6, 4, 7, 9, 1, 3]])

In [150]:
np.var([6,8,5,8,1,8,5,3,9,6])

5.6899999999999995

In [148]:
np.var(s, axis=0)

array([5.69, 8.61, 4.05, 6.56, 8.84, 5.56])

In [ ]:
ss, dd = np.random.randint(1,10,(10,3,3)), np.random.randint(1,10,(10,3,3))

In [137]:
np.sqrt(np.mean((np.square(s-d)), axis=0))

array([4.20713679, 4.07430976, 3.36154726, 4.35889894, 2.46981781,
       3.49284984])

In [181]:
a,b = torch.randn(size=(10,6)), torch.randn(size=(10,6))
ll(a,b)

tensor(2.0854)

In [180]:
torch.mean(torch.abs(a-b), axis=0).mean()

tensor(0.9820)

In [90]:
torch.randn(size=(5,3,3))[:, 1, :].size()

torch.Size([5, 3])

In [119]:
torch.manual_seed(0)
ll=torch.nn.MSELoss()
#ll(torch.randn(size=(10,3,3)), torch.randn(size=(10,3,3)), )
a,b = torch.randn(size=(10,3,3)), torch.randn(size=(10,3,3))
ll(a,b)

tensor(2.0054)

In [132]:
np.sqrt(((a-b)**2).mean(axis=0))

tensor([[1.1886, 1.4367, 1.1775],
        [1.4666, 1.5326, 1.9430],
        [1.8702, 0.6733, 0.9794]])

In [66]:
torch.randn(size=(10,6))

tensor([[ 1.0113, -0.0732,  0.0119, -1.4309, -1.1297,  0.3026],
        [ 0.3153, -2.2901,  1.3890, -2.3477,  0.3029,  0.7142],
        [-0.4341, -0.2742, -1.4934, -0.4970, -0.2888,  1.7183],
        [-1.7579, -0.1910, -0.9612,  1.4745, -1.5443,  0.3256],
        [ 3.0793, -0.5077,  0.8296, -0.6592,  0.8686, -0.3669],
        [ 0.3278, -1.3919, -1.6926,  0.1608,  0.9647, -0.7180],
        [-0.3258,  0.3614,  0.8608,  0.5456,  0.9697, -1.5581],
        [-0.7141,  0.9808, -0.0914, -0.0342, -0.4348,  1.0807],
        [-0.2383,  0.3174, -0.0461, -1.9251, -0.1943, -2.6019],
        [ 0.1466, -0.2510, -0.0317,  2.2110, -0.2807, -0.5661]])

In [11]:
asr = np.random.randint(1,10,(5,3))
asr

array([[2, 6, 2],
       [7, 7, 8],
       [4, 1, 9],
       [8, 6, 9],
       [1, 7, 6]])

In [19]:
akk=None

In [20]:
np.save(akk+'sha.npy', asr)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [18]:
np.load('ggsa.npy')

array([[2, 6, 2],
       [7, 7, 8],
       [4, 1, 9],
       [8, 6, 9],
       [1, 7, 6]])

In [6]:
[print(i) for i in tqdm.trange(20,33)]

100%|██████████| 13/13 [00:00<00:00, 145402.54it/s]

20
21
22
23
24
25
26
27
28
29
30
31
32


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [10]:
es = '122'
print(f"dsd{es}")

SyntaxError: f-string expression part cannot include a backslash (2445864702.py, line 2)

In [26]:
ll= np.load('3_lossData.npy', allow_pickle=True).item()

In [27]:
ll

{'tran_loss': array([6.9104128 , 1.72165208, 1.12056489]),
 'test_loss': 0.38774973154067993,
 'val_loss': array([7.92183709, 4.12310684, 0.78889355]),
 'min_loss': 1.1205648879210155,
 'Scores': {'Bias': tensor([[1.4915, 0.7079, 0.4573],
          [0.7411, 0.3098, 0.2950],
          [1.0243, 0.2743, 0.4294]]),
  'Var': tensor([[4.3387e-03, 7.3949e-05, 6.7337e-04],
          [2.0843e-03, 1.2376e-05, 8.3162e-05],
          [8.3426e-04, 1.0224e-04, 1.9757e-04]]),
  'Rmse': tensor([[1.8034, 0.7615, 0.5461],
          [0.8942, 0.3718, 0.3472],
          [1.1949, 0.3457, 0.4933]])}}

In [20]:
ll['Scores']['Bias']

tensor([0.3140, 0.2974, 0.4043, 0.3220, 0.8763, 0.4406])

In [2]:
def printer(c):
    print("in c: ", c)
    return 2*c

In [4]:
xy = printer

In [6]:
p=xy(55)

in c:  55


In [19]:
arr = torch.randint(1, 10, size=(4, 1, 6))
arr2 = torch.randint(1, 10, size=(4, 3))

In [3]:
torch.arange(start=1, end=10, step=1).reshape((3,3))

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [20]:
geo = torch.randint(1, 20, size=(15,))
rcoef = torch.randn((6,5))
print(geo.size(), rcoef.size())

torch.Size([15]) torch.Size([6, 5])


In [26]:
geo.reshape(5,3).unsqueeze(axis=2).size(), rcoef.reshape(5,3,2).size()

(torch.Size([5, 3, 1]), torch.Size([5, 3, 2]))

In [32]:
z = torch.empty(0)
torch.cat((z,geo.reshape(5,3,1)))

tensor([[[13.],
         [ 3.],
         [ 4.]],

        [[11.],
         [10.],
         [ 1.]],

        [[ 8.],
         [19.],
         [16.]],

        [[ 2.],
         [ 6.],
         [11.]],

        [[11.],
         [11.],
         [ 6.]]])

In [33]:
torch.cat((geo.reshape(5,3,1), rcoef.reshape(5,3,2)), dim=2)

tensor([[[ 1.3000e+01,  5.8685e-01, -2.6242e+00],
         [ 3.0000e+00,  1.1047e+00,  1.6436e-01],
         [ 4.0000e+00,  3.2146e-01,  4.9451e-03]],

        [[ 1.1000e+01, -6.5468e-01, -2.4775e-01],
         [ 1.0000e+01,  2.2140e-01,  3.7288e-01],
         [ 1.0000e+00, -1.1572e+00,  7.6960e-01]],

        [[ 8.0000e+00,  1.3235e+00,  2.0768e-01],
         [ 1.9000e+01,  5.0488e-01,  1.6955e+00],
         [ 1.6000e+01,  1.3042e+00,  9.7482e-01]],

        [[ 2.0000e+00,  4.2528e-01, -2.5267e-01],
         [ 6.0000e+00, -1.7478e+00,  5.4494e-01],
         [ 1.1000e+01,  1.5269e-01, -1.6250e+00]],

        [[ 1.1000e+01, -1.1076e+00, -3.2975e-01],
         [ 1.1000e+01, -5.5248e-01, -1.0446e+00],
         [ 6.0000e+00, -7.2621e-01,  5.6999e-01]]])

In [36]:
h = rir.generate(
    c=340,                                      # Sound velocity (m/s)
    fs=8000,                                    # Sample frequency (samples/s)
    r=np.sort([4.20, 2.50, 2.40]),               # Receiver position(s) [x y z] (m)
    s=np.sort([2.40, 1.30, 1.40]),                 # Source position [x y z] (m)
    L=[8.30, 5.00, 4.80],                        # Room dimensions [x y z] (m)
    nsample=4096,                               # Number of output samples
    beta=np.sort([0.24, 1.00, 0.66, 0.78, 0.38, 0.48])                    # Room reflection coefficients [x1 x2 y1 y2 z1 z2]
)
h

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       ...,
       [-1.01968700e-07],
       [ 8.04393176e-10],
       [-6.33972618e-08]])

In [108]:
np.isnan(h).any() == True

False

In [22]:
room_geo  = np.array([np.random.randint(low=60, high=100)/10, np.random.randint(low=50, high=80)/10, np.random.randint(low=40, high=60)/10])
# random room coeffs
room_coeff = np.random.uniform(size=(6))
# random source and receiver position
source_position = [np.random.randint(low=(len*10)/2-10, high=(len*10)/2+10)/10 for len in room_geo]
receiver_position = [np.random.randint(low=(len*10)/2-10, high=(len*10)/2+10)/10 for len in room_geo]
#
#print("Room geometry: ", room_geo, " receiver position: ", receiver_position, "source position: ", source_position, "room coeff: ", room_coeff)
# generate ISM data
h = rir.generate(
    c=340,                                      # Sound velocity (m/s)
    fs=8000,                                    # Sample frequency (samples/s)
    r=np.sort(receiver_position),               # Receiver position(s) [x y z] (m)
    s=np.sort(source_position),                 # Source position [x y z] (m)
    L=np.sort(room_geo),                        # Room dimensions [x y z] (m)
    nsample=4096,                               # Number of output samples
    beta=np.sort(room_coeff)                    # Room reflection coefficients [x1 x2 y1 y2 z1 z2]
)

# concat the geometry and reflection coefficients
h = np.concatenate((h.squeeze(), np.concatenate((room_geo, room_coeff))))
h = np.expand_dims(h, axis=1)
# save ISM data
print("ISM data: ", h.shape)

ISM data:  (4105, 1)


In [54]:
np.random.uniform(size=(6))

array([0.94823008, 0.55364852, 0.25966412, 0.98941496, 0.08847435,
       0.53749568])

In [51]:
np.isin(1.000, np.array([1, 2,3,4]))

array(True)

In [58]:
round(0.99578914, 2)

1.0

In [7]:
rer="123"

In [8]:
print(f"lind{rer}")

lind123


In [61]:
'%.2f'%(0.99578914)

'1.00'

In [250]:
np.random.seed(14)
print("pie", np.random.randint(low=1,high = 9))
count = 0
count2 = 0
for i in tqdm.trange(10000):
    room_geo  = [np.random.randint(low=60, high=100)/10, np.random.randint(low=50, high=80)/10, np.random.randint(low=40, high=60)/10]
  
    source_position = [np.random.randint(low=(len*10)/2-20, high=(len*10)/2+20)/10 for len in room_geo]
    receiver_position = []
    while True:
        receiver_position = [np.random.randint(low=(len*10)/2-20, high=(len*10)/2+20)/10 for len in room_geo]
        if (np.sort(source_position) == np.sort(receiver_position)).all():
            count += 1
            print(i, " receiver position: ", receiver_position, "source position: ", source_position)
            continue
        else:
            count2 += 1
            break

        #receiver_position = [np.random.randint(low=(len*10)/2-20, high=(len*10)/2+20)/10 for len in room_geo]
    #print(i,"Room geometry: ", room_geo, " receiver position: ", receiver_position, "source position: ", source_position, "room coeff: ", room_coeff)
print("count: ",count, count2)

pie 4


100%|██████████| 10000/10000 [00:00<00:00, 36120.24it/s]

9114  receiver position:  [4.9, 3.7, 2.9] source position:  [4.9, 3.7, 2.9]
count:  1 10000


In [2]:
s = np.array([2.8, 3.3, 1.9])   # org
r = np.array([1.9, 2.8, 3.3])   # same but scattered  
q = np.array([1.7, 2.8, 3.2])   # 1 ele same
t = np.array([1.7, 2.8, 3.3])   # 2 ele same

In [199]:
(np.sort(s) == np.sort(t))

array([False,  True,  True])

In [200]:
(np.sort(s) == np.sort(t)).all()

False

In [201]:
np.all(np.sort(s), np.sort(t))

TypeError: only integer scalar arrays can be converted to a scalar index

In [126]:
np.random.seed(0)
print(np.random.uniform(size=(6)))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411]


In [110]:
for i in tqdm.trange(10000):
    x = np.random.uniform(size=(6)).round(2)
    x[x==1.0] = 0.99
    y = x.copy()
    y[y==1.0] = 0.99
    if np.isin(1.0, x):
        print(i, x, y)

100%|██████████| 10000/10000 [00:00<00:00, 40526.67it/s]


In [3]:
torch.jit.script(Geometry_estimator())

RecursiveScriptModule(
  original_name=Geometry_estimator
  (convLayers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Conv_block
      (seq): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv1d)
        (1): RecursiveScriptModule(original_name=BatchNorm1d)
        (2): RecursiveScriptModule(original_name=ReLU)
      )
    )
    (1): RecursiveScriptModule(
      original_name=Conv_block
      (seq): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv1d)
        (1): RecursiveScriptModule(original_name=BatchNorm1d)
        (2): RecursiveScriptModule(original_name=ReLU)
      )
    )
    (2): RecursiveScriptModule(
      original_name=Conv_block
      (seq): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv1d)
        (1): RecursiveScriptModule(original_n

In [2]:
gg = torch.compile(Geometry_estimator())
gg

OptimizedModule(
  (_orig_mod): Geometry_estimator(
    (convLayers): Sequential(
      (0): Conv_block(
        (seq): Sequential(
          (0): Conv1d(1, 32, kernel_size=(4,), stride=(4,), bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (1): Conv_block(
        (seq): Sequential(
          (0): Conv1d(32, 32, kernel_size=(2,), stride=(2,), bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (2): Conv_block(
        (seq): Sequential(
          (0): Conv1d(32, 128, kernel_size=(8,), stride=(8,), bias=False)
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (3): Conv_block(
        (seq): Sequential(
          (0): Conv1d(128, 128, kernel_size=(2,), stride=(2,), bias=False)
          (1): Bat

In [12]:
room_geo

array([7.3, 5.1, 5.1])

In [65]:
dd = np.load("./rirData/ism_10k.npy")

In [66]:
data = pd.DataFrame(dd)
#tuples = []
#data.iloc[1][:9]

In [31]:
data.shape

(2, 4105)

In [61]:
xx, zz = train_test_split(dd, train_size=0.7, random_state=1)
len(xx), len(zz)

(28000, 12000)

In [64]:
40000/50

800.0

In [62]:
yy,az = train_test_split(zz, test_size=0.333333, random_state=1) 
len(yy), len(az)

(8000, 4000)

In [46]:
x,y,z=torch.utils.data.random_split(data, (280, 80, 40), torch.Generator().manual_seed(42))

In [50]:
x[1]

0     -0.000169
1      0.001994
2     -0.000006
3     -0.000049
4     -0.000208
         ...   
395    0.000474
396    0.006176
397    0.000136
398   -0.000053
399    0.000793
Name: 318, Length: 400, dtype: float64

In [34]:
data.iloc[0][:9]

0    8.30
1    5.00
2    4.80
3    0.24
4    1.00
5    0.66
6    0.78
7    0.38
8    0.48
Name: 0, dtype: float64

In [11]:
data = torch.tensor(data)


ValueError: expected sequence of length 100 at dim 0 (got 4105)

In [43]:
from utils.dataLoader import ISMDataset as ism

batches = torch.utils.data.DataLoader(ism(data=y), batch_size=5, shuffle=True, num_workers=4)

In [44]:
len(batches)

16

In [60]:
a,s=0, 0
for idx,b in enumerate(batches):
    a,s,c = b
    print(idx,"A",a.size(),"S", s.size(),"C", c.size())

0 A torch.Size([5, 3]) S torch.Size([5, 6]) C torch.Size([5, 4096])
1 A torch.Size([5, 3]) S torch.Size([5, 6]) C torch.Size([5, 4096])
2 A torch.Size([5, 3]) S torch.Size([5, 6]) C torch.Size([5, 4096])
3 A torch.Size([5, 3]) S torch.Size([5, 6]) C torch.Size([5, 4096])


In [19]:
for xc,data in tqdm.tqdm(enumerate(batches, 0), unit="batch", total=len(batches)):
    #print(xc)
    x=xc

100%|██████████| 20/20 [00:00<00:00, 51.97batch/s]


In [18]:
for batch_index, data in tqdm.tqdm(enumerate(batches), unit='batch',
                              total=len(batches),
                              desc ="My Progress Bar"):
    print(batch_index)

My Progress Bar:  40%|████      | 8/20 [00:00<00:00, 32.04batch/s]

0
1
2
3
4
5
6
7
8


My Progress Bar: 100%|██████████| 20/20 [00:00<00:00, 53.10batch/s]

9
10
11
12
13
14
15
16
17
18
19


In [31]:
a = a.cuda()

In [37]:
torch.randn((2,5,1)).reshape(2, -1).size()

torch.Size([2, 5])

In [33]:
a

tensor([[9.9000, 7.3000, 4.7000],
        [7.3000, 5.3000, 5.7000],
        [6.6000, 7.2000, 5.8000],
        [6.6000, 7.1000, 5.1000],
        [7.9000, 7.2000, 4.6000]], device='cuda:0', dtype=torch.float64)

In [11]:
tuples.append(data[:3])
tuples.append(data[3:9])
tuples.append(data[9:])

In [12]:
tuples[0]


,0,1,2,3,4,5,6,7,8,9,...,4095,4096,4097,4098,4099,4100,4101,4102,4103,4104
0,6.6,7.1,5.1,0.74,0.40,0.06,0.98,0.67,0.94,-0.000003,...,-7.382923e-06,-8.597558e-06,-6.104812e-06,-6.436633e-06,-5.621640e-06,-1.587070e-06,-8.767135e-06,9.240165e-06,1.006314e-05,-6.056572e-06
1,7.8,6.1,4.0,0.85,0.75,0.45,0.51,0.28,0.72,0.000123,...,-6.295699e-08,-1.326515e-07,-6.025048e-07,-3.875286e-07,-3.076428e-07,-3.986695e-07,-3.510979e-07,-2.832408e-07,-2.644297e-07,-2.569008e-07
2,7.3,5.3,5.7,0.03,0.56,0.75,0.01,0.87,0.47,-0.000358,...,-4.996883e-10,-9.963400e-10,2.173851e-10,-1.465849e-09,3.027487e-09,1.665801e-08,1.181978e-08,3.101104e-09,-6.473393e-09,1.209245e-08
